In [1]:
import numpy as np
import pandas as pd
from scipy.stats import mode

# função auxiliar chamada por 'my_knn'
# parâmetro 'instances' -> lista contendo dois dados (a função retorna a distância entre estes)
def get_distance(dataframe, instances, target): 

    df_aux = dataframe.drop(columns = [target])
    
    s = 0
    e = len(df_aux.columns) 
    for expl_var in df_aux.columns: 
        s += ((abs(instances[0][expl_var] - instances[1][expl_var])) ** e) 
        
    return s ** (1 / e)

# parâmetro 'task' -> especifica qual tarefa deve ser realizada ('regressao' / 'classificacao')
def my_knn(dataframe, new_instance, target, n_neighbors, task, est = None):
    
    distances = []
    for row in dataframe.index:
        distance = get_distance(dataframe, [new_instance, dataframe.iloc[row]], target)
        distances.append([distance, dataframe.iloc[row][target]])
    
    distances.sort()    
    closest = distances[:n_neighbors]
    
    closest_targets = []
    for i in closest:
        closest_targets.append(i[1])
        
    if task == 'classificacao':
        ret = mode(closest_targets)
        print('classe: {0}\nprobabilidade: {1}%'.format(ret[0][0], 
                                            round(ret[1][0] / len(closest_targets) * 100, 2)))
    
    if task == 'regressao':
        
        if est == 'media':    
            return(round(np.mean(closest_targets), 2), round(np.std(closest_targets), 2))
        
        if est == 'mediana':
            return(round(np.median(closest_targets), 2), round(np.std(closest_targets), 2))

In [2]:
from sklearn.datasets import load_boston, load_iris, load_diabetes, load_wine, load_breast_cancer
# CARREGANDO DATASETS, TRANSFORMANDO-OS EM DATAFRAMES
sk_dtsts = ['boston', 'iris', 'diabetes', 'wine', 'breast_cancer']
df_names = ['bstn', 'irs', 'dbts', 'wn', 'brst']

for dtst in sk_dtsts:
    exec("{0} = load_{0}()".format(dtst))
    
for load, name in zip(sk_dtsts, df_names):
    exec("{0} = pd.DataFrame({1}.data, columns = {1}.feature_names)".format(name, load))
    exec("{0}['target'] = pd.Series({1}.target)".format(name, load))
    
# bstn -> regressão
# irs -> classificação
# dbts -> regressão
# wn -> classificação
# brst -> classificação

In [3]:
my_knn(dataframe = bstn, new_instance = bstn.iloc[50], target = 'target', 
       n_neighbors = 10, task = 'regressao', est = 'mediana')

(30.55, 5.72)